In [29]:
import pandas as pd
import numpy as np
from keras.losses import MeanAbsoluteError, MeanSquaredError
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv('../data/processed/mbajk_dataset.csv')
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)

In [31]:
df.set_index('date', inplace=True)
df = df.resample('H').mean()
df.reset_index(inplace=True)
df.drop('date', inplace=True, axis=1)
df.dropna(inplace=True)

C:\Users\Tine Šuster\AppData\Local\Temp\ipykernel_16248\210845773.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').mean()


In [32]:
df.columns

Index(['temperature', 'dew_point', 'apparent_temperature', 'surface_pressure',
       'available_bike_stands', 'lagged_available_bike_stands',
       'rolling_mean_bike_stands', 'rolling_std_bike_stands',
       'diff_available_bike_stands', 'temperature_diff'],
      dtype='object')

In [33]:
def create_dataset(dataset, look_back=1, look_forward=5):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward):
        a = dataset.iloc[i:(i+look_back), :]
        dataX.append(a)
        
        dataY.append(dataset.iloc[i + look_back:i + look_back + look_forward]['available_bike_stands'])
    return np.array(dataX), np.array(dataY)

look_back = 1
look_forward = 7
dataX, dataY = create_dataset(df, look_back, look_forward)

In [34]:
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.2, random_state=1234)

In [35]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(Dense(100, activation='relu'))
model.add(LSTM(100, activation='relu'))
model.add(Dense(7))
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError(),metrics=[MeanSquaredError(), MeanAbsoluteError()])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 100)            44400     
                                                                 
 dense_6 (Dense)             (None, 1, 100)            10100     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_7 (Dense)             (None, 7)                 707       
                                                                 
Total params: 135,607
Trainable params: 135,607
Non-trainable params: 0
_________________________________________________________________


In [36]:
n_epochs = 100
# history = model.fit(trainX, trainY, epochs=n_epochs, batch_size=16, verbose=True)

In [37]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=MeanSquaredError(),metrics=[MeanSquaredError(), MeanAbsoluteError()])
history = model.fit(dataX, dataY, epochs=n_epochs, batch_size=16, verbose=True)

Epoch 1/100
146/146 [==============================] - 1s 2ms/step - loss: 0.3606 - mean_squared_error: 0.3606 - mean_absolute_error: 0.5248
Epoch 2/100
146/146 [==============================] - 0s 2ms/step - loss: 0.1268 - mean_squared_error: 0.1268 - mean_absolute_error: 0.2915
Epoch 3/100
146/146 [==============================] - 0s 2ms/step - loss: 0.0667 - mean_squared_error: 0.0667 - mean_absolute_error: 0.2119
Epoch 4/100
146/146 [==============================] - 0s 2ms/step - loss: 0.0613 - mean_squared_error: 0.0613 - mean_absolute_error: 0.2017
Epoch 5/100
146/146 [==============================] - 0s 2ms/step - loss: 0.0589 - mean_squared_error: 0.0589 - mean_absolute_error: 0.1964
Epoch 6/100
146/146 [==============================] - 0s 2ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.1926
Epoch 7/100
146/146 [==============================] - 0s 2ms/step - loss: 0.0556 - mean_squared_error: 0.0556 - mean_absolute_error: 0.1898
Epoch 8/100
1

In [38]:
from datetime import datetime

now = datetime.now() 
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

filename = f"../reports/{timestamp}_train_metrics.txt"

with open(filename, "w") as file:
    file.write("Epoch\t" + "\t".join(history.history.keys()) + "\n")
    for i in range(n_epochs):
        file.write(f"{i+1}\t" + "\t".join(str(history.history[metric][i]) for metric in history.history.keys()) + "\n")
        
test_metrics = model.evaluate(testX, testY, verbose=0)
test_filename = f"../reports/{timestamp}_metrics.txt"

with open(test_filename, "w") as file:
    file.write("Metric\tValue\n")
    for metric, value in zip(model.metrics_names, test_metrics):
        file.write(f"{metric}\t{value}\n")

In [40]:
model.save(f"../models/basic_model.h5")
# model.save(f"../models/basic_model")